In [1]:
from tensorflow.compat.v1 import ConfigProto
import os

import tensorflow as tf
from tensorflow import keras
from utilsSegmenter import *
from sklearn.metrics import confusion_matrix
import seaborn as sns
from datetime import datetime
import os
import pprint


# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, Conv1D, Convolution2D, Deconvolution2D, Cropping2D, UpSampling2D
from keras.layers import Input,  Conv2DTranspose
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers import Concatenate, concatenate, Reshape
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.layers import Input, merge
from keras.regularizers import l2
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout


creating network model using gpu 1


In [2]:
 class SegmenterDNNModel():
    def __init__(self, num_classes=10, batch_size=32, inputShape=(64,64,3), dropout_prob=0.25):
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.inputShape = inputShape
        self.dropout_prob = dropout_prob

    def conv2d_block(self, input_tensor, n_filters, kernel_size=3, batchnorm=True):
        # first layer
        x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
                   padding="same")(input_tensor)
        if batchnorm:
            x = BatchNormalization()(x)
        
        x = Activation("relu")(x)
        x = Dropout(dropout_prob)
        # second layer
        x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
                   padding="same")(x)
        if batchnorm:
            x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Dropout(dropout_prob)
        # Third layer
        x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
                   padding="same")(x)
        if batchnorm:
            x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Dropout(dropout_prob)


        return x

    def create_model(self):

        inputs = Input(shape=self.inputShape)

        down1 = Conv2D(32, (3, 3),padding='same')(inputs)
        down1 = Activation('relu')(down1)
        down1_pool = MaxPooling2D((2, 2), strides=(2, 2))(down1)
        down2 = Conv2D(64, (3, 3), padding='same')(down1_pool)
        down2 = Activation('relu')(down2)
        down2_pool = MaxPooling2D((2, 2), strides=(2, 2))(down2)
        
        
        down3 = Conv2D(128, (2, 2),padding='same')(down2_pool)
        down3 = Activation('relu')(down3)
        down3 = Conv2D(128, (2, 2),padding='same')(down3)
        down3 = Activation('relu')(down3)



        
        
        up1 = UpSampling2D((2, 2))(down3)
        up1 = concatenate([down2, up1], axis=3)
        up1 = Conv2D(256, (3, 3), padding='same')(up1)
        up1 = Activation('relu')(up1)

        up2 = UpSampling2D((2, 2))(up1)
        up2 = Conv2D(256, (3, 3), padding='same')(up2)
        up2 = Activation('relu')(up2)
  

        up3 = Conv2D(256, (3, 3), padding='same')(up2)
        up3 = Activation('relu')(up3)
        
        classify = Conv2D(self.num_classes, (1, 1), activation='sigmoid')(up3)

        model = Model(inputs=inputs, outputs=classify)
        model.summary()
        return model

"\n       inputs = Input(shape=self.inputShape)\n\n       down1 = Conv2D(32, (3, 3),padding='same')(inputs)\n       down1 = Activation('relu')(down1)\n       down1_pool = MaxPooling2D((2, 2), strides=(2, 2))(down1)\n       down2 = Conv2D(64, (3, 3), padding='same')(down1_pool)\n       down2 = Activation('relu')(down2)\n\n       \n       \n       up1 = UpSampling2D((2, 2))(down2)\n       up1 = concatenate([down1, up1], axis=3)\n       up1 = Conv2D(256, (3, 3), padding='same')(up1)\n       up1 = Activation('relu')(up1)\n\n       up2 = Conv2D(256, (3, 3), padding='same')(up1)\n       up2 = Activation('relu')(up2)\n\n       classify = Conv2D(self.num_classes, (1, 1), activation='sigmoid')(up2)\n\n       model = Model(inputs=inputs, outputs=classify)\n       model.summary()\n       return model\n"

In [3]:

##Create Model
image_shape = (cfg.IMAGE_HEIGHT, cfg.IMAGE_WIDTH, cfg.IMAGE_CHANNEL)
modelObj = SegmenterDNNModel(num_classes=cfg.NUM_CLASS, batch_size=cfg.BATCH_SIZE, inputShape=image_shape, dropout_prob=cfg.DROPOUT_PROB)
model = modelObj.create_model()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 128, 128, 32) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 64, 64, 32)   0           activation[0][0]                 
______________________________________________________________________________________________

In [4]:
#### show how the data looks like
show_statistics(cfg.training_data_dir, fineGrained=cfg.fineGrained, title=" Training Data Statistics ")
show_statistics(cfg.validation_data_dir, fineGrained=cfg.fineGrained, title=" Validation Data Statistics ")
show_statistics(cfg.testing_data_dir, fineGrained=cfg.fineGrained, title=" Testing Data Statistics ")


######################################################################
##################### Training Data Statistics #####################
######################################################################
total image number 	 10817
total class number 	 3['triangle', 'circular', 'square']
class triangle 	 3703 images of total 10817
class circular 	 3626 images of total 10817
class square 	 3488 images of total 10817
##############################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################

In [9]:
def get_loss_ww_weights(weights):
    # weights = [0.3,0.3,0.3,0.1]
    def wcce(t_segements, pred_segments):
        y_true = t_segements
        y_pred = pred_segments
        Kweights = K.constant(weights)
        if not K.is_tensor(y_pred): y_pred = K.constant(y_pred)
        y_true = K.cast(y_true, y_pred.dtype)
        #return keras.categorical_crossentropy(y_true, y_pred) * K.sum(y_true * Kweights, axis=-1)
    return wcce

def get_loss_w_weights(y_true, y_pred, ratios):
    print(type(y_true))
    print((y_true.shape))
    print((y_pred.shape))

    print(tf.constant(ratios))
    cce = tf.keras.losses.categorical_crossentropy
    weights = tf.keras.metrics.categorical_crossentropy(t_segments, pred_segments)
    with_losses = cce(y_true, y_pred,sample_weight=weights)
    print(tf.constant(with_losses))
    print(cce)
    #ratios =  [0.04131836793275851, 0.041082430530561466, 0.05517578736268465, 0.8624234141739954]
    return with_losses


ratios = [0.04131836793275851, 0.041082430530561466, 0.05517578736268465, 0.8624234141739954] #The ratio of each class
opt = tf.optimizers.Adam(0.005)
model.compile(loss=tf.keras.losses.categorical_crossentropy
,optimizer= opt, metrics=['accuracy'])


In [10]:
#Train the model
history = model.fit_generator(generator = aug_train_batch_generator,
          epochs=cfg.NUM_EPOCHS,
          verbose=1,
          steps_per_epoch=(nbr_train_data//cfg.BATCH_SIZE), # total batch number
          validation_steps=(nbr_valid_data // cfg.BATCH_SIZE), # total batch number
          validation_data=valid_batch_generator,
          callbacks=[TensorBoard(log_dir="logs/{}_{}".format(cfg.NUM_EPOCHS,cfg.BATCH_SIZE), write_graph=True, write_images=False,
                                 histogram_freq=0)])

Instructions for updating:
Please use Model.fit, which supports generators.
  6/338 [..............................] - ETA: 4:51 - loss: 1.6361 - accuracy: 0.3366

KeyboardInterrupt: 

In [ ]:
t_data, t_segments = next(test_batch_generator)
pred_segments = model.predict(t_data, batch_size=cfg.BATCH_SIZE)
plot_sample_segmentation_results(t_data, t_segments, pred_segments, test_acc)
weights = tf.keras.metrics.categorical_crossentropy(t_segments, pred_segments)